<a href="https://colab.research.google.com/github/tjelinek/UniBern/blob/main/UniBern_Programming_Jelinek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1

In [ ]:
!pip install livelossplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 58.5 MB/s 


In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from livelossplot import PlotLossesKerasTF

from PIL import Image
import numpy as np

In [ ]:
devices = tf.config.list_physical_devices("GPU")
print(devices)
# tf.config.experimental.set_memory_growth(devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Preparing the dataset
As the first step, we prepare the dataset; we use imagenette, which we download using the wrapper 'tfds' of 'tf.data' functions. Because the images are of different sizes, we resize them to 600x600 and convert them to grayscale.

In [ ]:
(ds_train, ds_valid), ds_info = tfds.load('imagenette', split=['train', 'validation'], with_info=True, shuffle_files=True, as_supervised=True)
ds_test, ds_test_info = tfds.load('imagenet_v2', split='test', with_info=True, shuffle_files=True, as_supervised=True)

# Use the first 5000 examples
ds_train = ds_train.take(5000)
ds_valid = ds_valid.take(1000)
ds_test = ds_test.take(1000)

def preprocess(image, label):
    i = image
    i = tf.cast(i, tf.float32)
    i = tf.image.resize_with_crop_or_pad(i, 600, 600)
    i = tf.image.rgb_to_grayscale(i)
    return i, label

# We now pre-process the input - resize the images to a common size, and convert the images to grayscale.
ds_train = ds_train.map(preprocess)
ds_valid = ds_valid.map(preprocess)
ds_test = ds_test.map(preprocess)


fig = tfds.show_examples(ds_train, ds_info)

# Normalize the images so that the values lie in interval [0, 1].
def normalize(image, label):
    i = image
    i = tf.cast(i, tf.float32) * (1/255.0)
    return i, i

ds_train = ds_train.map(normalize).batch(32)
ds_valid = ds_valid.map(normalize).batch(32)
ds_test = ds_test.map(normalize).batch(32)




KeyboardInterrupt: ignored

In [ ]:

def get_encoder_conv_block(inputs, filters):
    """
    It is more efficient to replace a 5x5 convolution with a 3x3 convolution:
    C. Szegedy, V. Vanhoucke, S. Ioffe, J. Shlens and Z. Wojna, "Rethinking the Inception Architecture for Computer Vision,"
    2016 IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2016, pp. 2818-2826, doi: 10.1109/CVPR.2016.308.

    :param inputs: Previous layer.
    :param filters: Number of filters in the convolution.
    :return: The last layer of the block.
    """
    x = keras.layers.Conv2D(filters=filters, kernel_size=3, strides=(1, 1), padding='same', activation='relu')(inputs)
    x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)
    x = keras.layers.Conv2D(filters=filters, kernel_size=3, strides=(1, 1), padding='same', activation='relu')(x)
    x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)
    x = keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2))(x)

    return x

def get_decoder_conv_block(inputs, filters):
    """
    Decoder variant of the previous.

    :param inputs: Previous layer.
    :param filters: Number of filters in the convolution.
    :return: The last layer of the block.
    """
    # x = keras.layers.Activation("relu")(inputs)
    # x = keras.layers.Conv2DTranspose(filters=filters, kernel_size=3, strides=(1, 1), padding='same', activation='relu')(x)
    # x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)
    # x = keras.layers.Activation("relu")(x)
    x = keras.layers.Conv2DTranspose(filters=filters, kernel_size=3, strides=(2, 2), padding='same', activation='relu')(inputs)
    x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)
    # x = keras.layers.UpSampling2D(2)(x)

    return x

inputs = keras.Input(shape=(600, 600, 1))

inputs = keras.layers.Lambda(lambda x: x / 255.0)(inputs)

filt = 8
e1 = get_encoder_conv_block(inputs, 4)

e2 = get_encoder_conv_block(e1, 4)
e3 = get_encoder_conv_block(e2, 8)
e4 = get_encoder_conv_block(e3, 8)
e5 = get_encoder_conv_block(e4, 16)
e6 = get_encoder_conv_block(e5, 16)

m1 = keras.layers.Flatten()(e6)
m2 = keras.layers.Dense(10*10*16)(m1)
m3 = keras.layers.Reshape((10, 10, 16))(m2)

d0 = get_decoder_conv_block(m3, 16)
d0 = get_decoder_conv_block(d0, 16)

d1 = get_decoder_conv_block(d0, 32)
d1_cropped = tf.keras.layers.Cropping2D(cropping=((5, 0),(5, 0)))(d1)

d2 = get_decoder_conv_block(d1_cropped, 32)
r2 = keras.layers.Conv2D(32, 1, padding="same")(e2)
# d2 = keras.layers.add([d2, r2])

d3 = get_decoder_conv_block(d2, 16)
r3 = keras.layers.Conv2D(16, 1, padding="same")(e1)
# d3 = keras.layers.add([d3, d3])

d4 = get_decoder_conv_block(d3, 8)
d4 = keras.layers.Lambda(lambda x: x * 255.0)(d4)
output = keras.layers.Conv2D(filters=1, kernel_size=1, activation='sigmoid')(d4)

autoencoder = keras.models.Model(inputs, output)
autoencoder.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=1e-2))


autoencoder.summary()


In [ ]:
autoencoder.fit(ds_train.repeat(),
                epochs=50,
                steps_per_epoch=5000 // 32,
                verbose=1)


In [ ]:
y_pred = autoencoder.predict(ds_test)

In [ ]:
imgs_orig, _ = next(iter(ds_test))

imgs_pred = autoencoder.predict(imgs_orig)

imgs_orig = imgs_orig * 255.0
imgs_orig = np.uint8(imgs_orig).squeeze(axis=3)

imgs_pred = imgs_pred * 255.0
imgs_pred = np.uint8(imgs_pred).squeeze(axis=3)

In [ ]:
from matplotlib import pyplot as plt
fig = plt.gcf()
fig.set_size_inches(10, 10*32)

plt.imshow(np.concatenate([np.concatenate([imgs_orig[i], imgs_pred[i]], axis = -1, ) for i in range(imgs_orig.shape[0])]), cmap = 'gray')
plt.show()

# Task 2

In [ ]:
(ds_train1, ds_test1), ds_info = tfds.load('cifar10', split=['train', 'test'], with_info=True, shuffle_files=False, as_supervised=True)
(ds_train2, ds_test2), ds_info = tfds.load('cifar10', split=['train', 'test'], with_info=True, shuffle_files=False, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incompleteGBY4QF/cifar10-train.tfrecord*...:   0%|          | 0/…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incompleteGBY4QF/cifar10-test.tfrecord*...:   0%|          | 0/1…

Dataset cifar10 downloaded and prepared to ~/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
def normalize(image, label):
    i = image
    i = tf.cast(i, tf.float32) * (1/255.0)
    return i

ds_train1 = ds_train1.take(1000).map(normalize)
ds_train2 = ds_train2.take(1000).map(normalize)

l1 = 0
l2 = 0

def label(i1, i2):
  global l1
  global l2

  output = (i1, i2), (tf.one_hot(l1, depth=1000), tf.one_hot(l2, depth=1000))
  # output = (i1, l1), (i2, l2)
  l1 += 1
  if l1 >= 1000:
    l1 = 0
    l2 += 1
  return output

# ds_train1 = ds_train1.shuffle(buffer_size=1000, seed=1, reshuffle_each_iteration=True).repeat()
# ds_train2 = ds_train2.shuffle(buffer_size=1000, seed=2, reshuffle_each_iteration=True).repeat()

ds_train = tf.data.Dataset.zip((ds_train1, ds_train2))
ds_train = ds_train.map(label)

ds_train = ds_train.batch(32).repeat()

ds_test1 = ds_test1.map(normalize)
ds_test2 = ds_test2.map(normalize)

ds_test1 = ds_test1.shuffle(buffer_size=1000, seed=1, reshuffle_each_iteration=True)
ds_test2 = ds_test2.shuffle(buffer_size=1000, seed=2, reshuffle_each_iteration=True)

ds_test = tf.data.Dataset.zip((ds_test1, ds_test2))

ds_test = ds_test.map(label)


In [ ]:
inputs = [keras.Input(shape=(32, 32, 3)), keras.Input(shape=(32, 32, 3))]

# For convenience, I add the images in the network (and omit dividing by 0.5).
# Since the add layer has no trainable parameters, it is equivalent to computing the
# average prior to feeding it as an input.
x = keras.layers.Add()(inputs)

x = keras.layers.Conv2D(filters = 4, kernel_size=3, padding='same', strides=2, activation='relu')(x)
x = keras.layers.Conv2D(filters = 8, kernel_size=3, padding='same', strides=2, activation='relu')(x)
x = keras.layers.Conv2D(filters = 16, kernel_size=3, padding='same', strides=2, activation='relu')(x)
# x = keras.layers.Conv2D(filters = 32, kernel_size=3, padding='same', strides=2, activation='relu')(x)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(4*4*2*16)(x)
x = keras.layers.Reshape((8, 4, 16))(x)

# Cropping causes the 
x1 = keras.layers.Cropping2D(cropping=((4, 0), (0, 0)))(x)
x2 = keras.layers.Cropping2D(cropping=((0, 4), (0, 0)))(x)

def get_decoder_branch(x):
  # for i in range(3):
  #   x = keras.layers.Conv2DTranspose(filters=16 / (2 ** i), kernel_size=3, strides=(2, 2), padding='same', activation='relu')(x)
  # x = x = keras.layers.Conv2D(filters = 3, kernel_size=1, padding='same', strides=1, activation='relu')(x)
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(1000, activation='softmax')(x)
  return x

out1 = get_decoder_branch(x1)
out2 = get_decoder_branch(x2)


model = keras.models.Model(inputs, [out1, out2])
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3))

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 add (Add)                      (None, 32, 32, 3)    0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 4)    112         ['add[0][0]']                

In [ ]:
model.fit(x=ds_train,
                epochs=100,
                steps_per_epoch=5000 // 32,
                verbose=1)


Epoch 1/100
156/156 [==============================] - 11s 12ms/step - loss: 0.8290 - dense_1_loss: 0.4113 - dense_2_loss: 0.4177
Epoch 2/100
156/156 [==============================] - 2s 12ms/step - loss: 1.2234e-10 - dense_1_loss: 9.7873e-11 - dense_2_loss: 2.4468e-11
Epoch 3/100
156/156 [==============================] - 2s 11ms/step - loss: 1.7128e-10 - dense_1_loss: 1.2234e-10 - dense_2_loss: 4.8936e-11
Epoch 4/100
156/156 [==============================] - 2s 12ms/step - loss: 2.2021e-10 - dense_1_loss: 1.4681e-10 - dense_2_loss: 7.3405e-11
Epoch 5/100
156/156 [==============================] - 2s 12ms/step - loss: 1.4681e-10 - dense_1_loss: 7.3405e-11 - dense_2_loss: 7.3405e-11
Epoch 6/100
156/156 [==============================] - 2s 12ms/step - loss: 1.9575e-10 - dense_1_loss: 1.2234e-10 - dense_2_loss: 7.3405e-11
Epoch 7/100
156/156 [==============================] - 2s 12ms/step - loss: 1.9575e-10 - dense_1_loss: 1.2234e-10 - dense_2_loss: 7.3405e-11
Epoch 8/100
156/156 [===

In [ ]:
y_pred = model.predict(ds_test.batch(32))

In [ ]:
batch_size = 32

imgs_orig, _ = next(iter(ds_test.batch(batch_size)))
imgs_pred = model.predict(imgs_orig)

print(img_orig())

origs1 = [np.uint8(imgs_orig[0][i] * 255.0) for i in range(batch_size)]
origs2 = [np.uint8(imgs_orig[1][i] * 255.0) for i in range(batch_size)]

origs_conc = [np.concatenate([origs1[i], origs2[i]], axis=-2) for i in range(batch_size)]
origs_avg = [np.uint8((0.5*origs1[i] + 0.5*origs2[i])) for i in range(batch_size)]

preds1 = [np.uint8(imgs_pred[0][i] * 255.0) for i in range(batch_size)]
preds2 = [np.uint8(imgs_pred[1][i] * 255.0) for i in range(batch_size)]

preds_conc = [np.concatenate([preds1[i], preds2[i]], axis=-2) for i in range(batch_size)]



NameError: ignored

In [ ]:
from matplotlib import pyplot as plt
fig = plt.gcf()
fig.set_size_inches(5, 5*32)

plt.axis("off")
plt.imshow(np.concatenate([np.concatenate([origs_conc[i], origs_avg[i], preds_conc[i]], axis = -2, ) for i in range(batch_size)]))
plt.show()